In [ ]:
#LOGAN KELSCH
#TEST NN 1

#IMPORT LIBRARIES-------------------------------------------------------

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from keras.optimizers import SGD
from keras.initializers import GlorotUniform

tf.config.experimental.set_memory_growth

#LOAD DATA FROM CSV-------------------------------------------------------
# 3.0 collected data features:
# Symbol: /ES:XCME
# Work Time: 4/10/23, 3:05 PM - 9/13/24, 1:55 PM
# fulltime- seconds since 1970 (for appending data)
# FullK-    fullK plot for Stochastic of past hour
# diffKD-   value difference between fullK/fullD of past hour
# OB , OS-  boolean if Stoch plots are in top/btm 20% of range
# vol-      trade volume of each bar
# s15,30,60-Current price/time slope of past n minutes 
#ToD-       Time of Day in seconds
#perc30,60- percentile of low and high of past 30,60 mins and day
#RSI-       RSI output of past hour
#Wpercent-  William Percent output of past hour
#acc-       Rate of acceleration of past hour over half hours
################################
#DESIRED OUTPUTS
#Are formulated by making a bull and bear 0-1 oscillator
#standard deviations derived from upcoming 4 hours of movement
#Value description:
#If value==0
#  the conditions are not appealing for next n minutes
#If value==1
#  the price moves >=1 std. dev. in that direciton for next n mins
#else
#  price moves y std. dev. in that direction for next n minutes
#bull/bear15 - 15 minute direction lookahead
#bull/bear30 - 30 minute direction lookahead
#bull/bear60 - 60 minute direction lookahead

# Load the dataset
data = pd.read_csv('ES_5m_4-10-23_to_9-13-24.csv')

#testing random feature drops
#data = data.drop(columns='perc30')
#data = data.drop(columns='perc60')
#data = data.drop(columns='percD')


#TEMP DROP PRE-DUAL-OUTPUT NN
#data = data.drop(columns='bull15')
data = data.drop(columns='bear15')

data = data.drop(columns='bull30')
data = data.drop(columns='bear30')

data = data.drop(columns='bull60')
data = data.drop(columns='bear60')

# Separate features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

#PROCESS THE DATA-------------------------------------------------------

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#BUILD THE NEURAL NETWORK MODEL-------------------------------------------------------

opt1 = SGD(learning_rate=0.01)
opt2  = tf.keras.optimizers.Adam(clipnorm=0.8)

def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation='relu', kernel_initializer=GlorotUniform()),
        tf.keras.layers.Dense(256, activation='relu'),  
        tf.keras.layers.Dropout(0.9),
        tf.keras.layers.Dense(256, activation='relu'),  
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(256, activation='relu'),  
        tf.keras.layers.Dropout(0.9),
        tf.keras.layers.Dense(256, activation='relu'),  
        tf.keras.layers.Dropout(0.9),
        tf.keras.layers.Dense(16, activation='sigmoid', kernel_initializer=GlorotUniform()),       
        tf.keras.layers.Dense(1)  # Output layer for regression
    ])
    
    rmse='root_mean_squared_error'

    model.compile(optimizer=opt2, loss='binary_crossentropy', metrics=['mae','R2Score',rmse])
    return model

#TRAIN THE MODEL WITH CUSTOMIZABLE EPOCHS-------------------------------------------------------

epochs = 50

model = build_model()
history = model.fit(X_train, y_train, epochs=epochs, validation_split=0.5, verbose=1, shuffle=True)

#EVALUATE THE MODEL AND VISUALIZE RESULTS-------------------------------------------------------

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")

#R and R^2 calculation attempt in the 3 lines below this

#unexplained_error = tf.reduce_sum(tf.square(tf.sub(y, y_pred)))
#total_error = tf.reduce_sum(tf.square(tf.sub(y, tf.reduce_mean(y))))
#R_squared = tf.sub(1, tf.div(unexplained_error, total_error))

#print(f"R^2: {R_squared}")

# ^ need to figure out how to fit a "prediction" variable into first line of this calculation
#or just use a tensorflow_addon rsquared formula to calculate this

#online source suggests that loss < .22 and R^2 over .9 is good
#on a good pace for truly building something here

#confirming X and Y features post training
Xfeatures = data.columns[:-1]
Yfeatures = data.columns[-1]
print("TESTED FEATURES: ")
print(Xfeatures)
print("TESTING FOR: ")
print(Yfeatures)

#example prediction outputs, currently first 10 of training set
preds = model.predict(X_train)
print(preds[0:5])
print(preds[10000:10005])

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# Save the model
model.save('my_model.keras')
# Load the model
loaded_model = tf.keras.models.load_model('my_model.keras')